## Goal(s)
### Cluster subject lines and test out their CTR using online tools (if available)
#### 1. Different ways of embedding sentences
##### a. Average word2vec
##### b. word2vec -> TFIDF (multiply word2vec and tfidf representations of words and take average)
##### c. Pre-trained FastText
##### d. Self-trained FastText

##### Experiment with stop words, proper nouns etc

In [433]:
import bs4 as bs
import urllib.request
import re
import nltk
import pandas as pd
import gensim.downloader as api
import json
import numpy as np
import gensim.models
from numpy import dot
from numpy.linalg import norm

In [273]:
info = api.info()
for model_name, model_data in sorted(info['models'].items()):
    print(
        '%s (%d records): %s' % (
            model_name,
            model_data.get('num_records', -1),
            model_data['description'][:70] + '...',
        )
    )

__testing_word2vec-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Word vecrors of the movie matrix....
conceptnet-numberbatch-17-06-300 (1917247 records): ConceptNet Numberbatch consists of state-of-the-art semantic vectors (...
fasttext-wiki-news-subwords-300 (999999 records): 1 million word vectors trained on Wikipedia 2017, UMBC webbase corpus ...
glove-twitter-100 (1193514 records): Pre-trained vectors based on  2B tweets, 27B tokens, 1.2M vocab, uncas...
glove-twitter-200 (1193514 records): Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncase...
glove-twitter-25 (1193514 records): Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncase...
glove-twitter-50 (1193514 records): Pre-trained vectors based on 2B tweets, 27B tokens, 1.2M vocab, uncase...
glove-wiki-gigaword-100 (400000 records): Pre-trained vectors based on Wikipedia 2014 + Gigaword 5.6B tokens, 40...
glove-wiki-gigaword-200 (400000 records): Pre-trained vectors based on Wikipedi

In [277]:
for corpus_name, corpus_data in sorted(info['corpora'].items()):
    print(
#         '%s (%d records): %s' % (
        '%s (%d records)' % (
            corpus_name,
            corpus_data.get('num_records', -1),
#             corpus_data['description'][:70] + '...',
        )
    )

20-newsgroups (18846 records)
__testing_matrix-synopsis (-1 records)
__testing_multipart-matrix-synopsis (-1 records)
fake-news (12999 records)
patent-2017 (353197 records)
quora-duplicate-questions (404290 records)
semeval-2016-2017-task3-subtaskA-unannotated (189941 records)
semeval-2016-2017-task3-subtaskBC (-1 records)
text8 (1701 records)
wiki-english-20171001 (4924894 records)


In [341]:
info['corpora']['text8']

{'num_records': 1701,
 'record_format': 'list of str (tokens)',
 'file_size': 33182058,
 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py',
 'license': 'not found',
 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.',
 'checksum': '68799af40b6bda07dfa47a32612e5364',
 'file_name': 'text8.gz',
 'read_more': ['http://mattmahoney.net/dc/textdata.html'],
 'parts': 1}

In [317]:
# sentences = api.load('wiki-english-20171001')
### Too big dataset; will download later

In [393]:
def create_update_dataset():
    #Import data
    emails = pd.read_excel(r"C:\Users\Harsh Garg\Desktop\intract\Email Subject Line Scrapper.xlsx",'Saved Data')

    #Delete irrelevant columns and add relevant columns
    emails = emails[['Subject Line']]
    
    
    return list(emails['Subject Line'])

#### Adding emails to text8 vocabulary because word2vec cannot handle OOV words

In [415]:
new_sentences = create_update_dataset()
for index, sentence in enumerate(new_sentences):
    new_sentences[index] = sentence.split()

sentences = api.load('text8')

In [416]:
for sentence in sentences:
    new_sentences = new_sentences + sentence

In [426]:
output = []
def reemovNestings(l):
    for i in l:
        if type(i) == list:
            reemovNestings(i)
        else:
            output.append(i)

In [ ]:
reemovNestings(new_sentences)

In [443]:
output = output + ['fear']
output = output + ['anticipation']
output = output + ['joy']
output = output + ['trust']
output = output + ['pride']

In [447]:
#Takes A LOT of time to train :(

model = gensim.models.Word2Vec(sentences=output,vector_size=100, window=2, min_count=1, workers=4)
model.train(output, total_examples=len(output), epochs=1)

(28135312, 166099895)

In [430]:
def abs_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += abs(value - centroid[index])
    return dist

def square_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += (value - centroid[index])**2
    return dist

In [431]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [450]:
model.wv['summary']

KeyError: "Key 'summary' not present"

In [432]:
def data_preprocess_and_preds(pred_model):
    
    #Import data
    emails = pd.read_excel(r"C:\Users\Harsh Garg\Desktop\intract\Email Subject Line Scrapper.xlsx",'Saved Data')

    #Delete irrelevant columns and add relevant columns
    emails = emails[['Subject Line']]
    
    emails['Vector'] = ''

    emails['fear'] = ''

    emails['anticipation'] = ''

    emails['joy'] = ''

    emails['trust'] = ''

    emails['pride'] = ''

    emails['prediction'] = ''

    #Will be required for using a 'Switch' function later on
    dictionary = {0:'fear',
                 1:'anticipation',
                 2:'joy',
                 3:'trust',
                 4:'pride'}
    
    #to store vectors of the words of emails for k-means later
    vectors = pd.DataFrame()
    
    #Predictions
    for index,line in enumerate(emails['Subject Line']):
        words = line.split()
        vector_word = [0]*100
        num_words = 0
        for word in words:
            try:
                vector_word += pred_model[word]
                num_words+=1
            except:
                print(word + " not in my limited vocabulary")
        if num_words==0:
            pass
        else:
            vector_word /= num_words
        emails['Vector'].loc[index] = vector_word
        vectors = pd.concat([vectors, pd.DataFrame(vector_word).T])
        probs = []
        
        probs.append(dot(vector_word, model.wv['fear'])/(norm(vector_word)*norm(model.wv['fear'])))
#         probs.append(pred_model.similarity(words,'fear'))
        probs.append(dot(vector_word, model.wv['anticipation'])/(norm(vector_word)*norm(model.wv['anticipation'])))
#         probs.append(pred_model.similarity(words,'anticipation'))
        probs.append(dot(vector_word, model.wv['joy'])/(norm(vector_word)*norm(model.wv['joy'])))
#         probs.append(pred_model.similarity(words,'joy'))
        probs.append(dot(vector_word, model.wv['trust'])/(norm(vector_word)*norm(model.wv['trust'])))
#         probs.append(pred_model.similarity(words,'trust'))
        probs.append(dot(vector_word, model.wv['pride'])/(norm(vector_word)*norm(model.wv['pride'])))
#         probs.append(pred_model.similarity(words,'pride'))

        probs = softmax(probs)
        emails['fear'].loc[index] = probs[0]
        emails['anticipation'].loc[index] = probs[1]
        emails['joy'].loc[index] = probs[2]
        emails['trust'].loc[index] = probs[3]
        emails['pride'].loc[index] = probs[4]

        emails['prediction'].loc[index] = np.where(probs == max(probs))[0][0]
    emails['prediction'] = emails['prediction'].apply(lambda x: dictionary.get(x))

    vectors = vectors.reset_index(drop = True)

    emails['prediction'].value_counts()

    from sklearn.cluster import KMeans
    n_clusters = 15
    clf = KMeans(n_clusters=n_clusters, max_iter=100, init='k-means++', n_init=1)
    labels = clf.fit_predict(vectors)
    centroids = clf.cluster_centers_

    emails['Cluster'] = labels
    emails['abs_dist'] = ''
    for index, vect in enumerate(emails['Vector']):
        cluster = emails['Cluster'].loc[index]
        centroid = centroids[cluster]
        emails['abs_dist'].loc[index] = abs_distance(vect, centroid)

    min_indices = []
    for x in range(0,15):
        min_indices.append(emails[emails['abs_dist']==min(emails[emails['Cluster']==x]['abs_dist'])].index)
    
    return emails, vectors

In [422]:
mails, vecs = data_preprocess_and_preds(model)

Aey not in my limited vocabulary
Raju, not in my limited vocabulary
Salary not in my limited vocabulary
Aa not in my limited vocabulary
Gayi not in my limited vocabulary
Rey not in my limited vocabulary
Baba not in my limited vocabulary
🤓 not in my limited vocabulary


AttributeError: 'Word2Vec' object has no attribute 'n_similarity'

### FastText

#### Pre-trained

In [452]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model = FastText(vector_size=100)

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

wv = model.wv

In [453]:
wv.key_to_index

{'the': 0,
 'to': 1,
 'of': 2,
 'in': 3,
 'and': 4,
 'a': 5,
 'is': 6,
 'for': 7,
 'The': 8,
 'on': 9,
 'he': 10,
 'has': 11,
 'says': 12,
 'was': 13,
 'have': 14,
 'that': 15,
 'be': 16,
 'are': 17,
 'will': 18,
 'with': 19,
 'Mr': 20,
 'said.': 21,
 'at': 22,
 'from': 23,
 'by': 24,
 'been': 25,
 'not': 26,
 'as': 27,
 'his': 28,
 'an': 29,
 'it': 30,
 'were': 31,
 'had': 32,
 'after': 33,
 'but': 34,
 'they': 35,
 'said': 36,
 'who': 37,
 'this': 38,
 'Australian': 39,
 'we': 40,
 'Palestinian': 41,
 'their': 42,
 'which': 43,
 'people': 44,
 'two': 45,
 'there': 46,
 'up': 47,
 'about': 48,
 'also': 49,
 'its': 50,
 'out': 51,
 'South': 52,
 'into': 53,
 'would': 54,
 'US': 55,
 'when': 56,
 'against': 57,
 'first': 58,
 'New': 59,
 'more': 60,
 'last': 61,
 'I': 62,
 'He': 63,
 'A': 64,
 'Israeli': 65,
 'Australia': 66,
 'if': 67,
 'one': 68,
 'over': 69,
 'United': 70,
 'Government': 71,
 'or': 72,
 'than': 73,
 'no': 74,
 'all': 75,
 'could': 76,
 'three': 77,
 'before': 78,
 't

In [456]:
fear = wv['fear']
anticipation = wv['anticipation']
joy = wv['joy']
trust = wv['trust']
pride = wv['pride']

In [457]:
emails = pd.read_excel(r"C:\Users\Harsh Garg\Desktop\intract\Email Subject Line Scrapper.xlsx",'Saved Data')

In [458]:
emails.head(3)

,Date,Message Id,From,Email,Subject Line,Concated Body,Email from,Unnamed: 7
0,2021-10-01 19:26:52,17c3c24c23506d3c,Paytm Offers,noreply@paytmoffers.in,"Aey Raju, Salary Aa Gayi Rey Baba 🤓",Paytm par deals check kar to,NaN,NaN
1,2021-10-01 13:50:21,17c3af0a22235012,Swiggy Instamart,alerts@updates.swiggy.in,October's first grocery bill will be a short o...,Email Template \n[image: Get 3,NaN,NaN
2,2021-10-01 11:43:23,17c3a816c53f6b1c,Zomato,noreply@mailers.zomato.com,Guess who always makes you smile? 😃,Here are your hints - you lov,abhishek12318@gmail.com,Subject: Guess who always makes you smile? 😃


In [459]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

emails = emails[['Subject Line']]

emails['Vector'] = ''

emails['fear'] = ''

emails['anticipation'] = ''

emails['joy'] = ''

emails['trust'] = ''

emails['pride'] = ''

emails['prediction'] = ''

In [460]:
emails.head(1)

,Subject Line,Vector,fear,anticipation,joy,trust,pride,prediction
0,"Aey Raju, Salary Aa Gayi Rey Baba 🤓",,,,,,,


In [461]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [462]:
dictionary = {0:'fear',
             1:'anticipation',
             2:'joy',
             3:'trust',
             4:'pride'}

In [463]:
vectors = pd.DataFrame()

In [464]:
for index,line in enumerate(emails['Subject Line']):
    words = line.split()
    vector_word = [0]*100
    for word in words:
        vector_word += wv[word]
    vector_word /= len(words)
    emails['Vector'].loc[index] = vector_word
    vectors = pd.concat([vectors, pd.DataFrame(vector_word).T])
    probs = []
    probs.append(wv.n_similarity(words,['fear']))
    probs.append(wv.n_similarity(words,['anticipation']))
    probs.append(wv.n_similarity(words,['joy']))
    probs.append(wv.n_similarity(words,['trust']))
    probs.append(wv.n_similarity(words,['pride']))
    probs = softmax(probs)
    emails['fear'].loc[index] = probs[0]
    emails['anticipation'].loc[index] = probs[1]
    emails['joy'].loc[index] = probs[2]
    emails['trust'].loc[index] = probs[3]
    emails['pride'].loc[index] = probs[4]
    
    emails['prediction'].loc[index] = np.where(probs == max(probs))[0][0]

emails['prediction'] = emails['prediction'].apply(lambda x: dictionary.get(x))

In [465]:
emails.head()

,Subject Line,Vector,fear,anticipation,joy,trust,pride,prediction
0,"Aey Raju, Salary Aa Gayi Rey Baba 🤓","[-0.029583398674731143, 0.04560574136849027, -...",0.200064,0.200065,0.199753,0.200059,0.200058,anticipation
1,October's first grocery bill will be a short o...,"[-0.203917137067765, 0.3200864117126912, -0.37...",0.200053,0.200059,0.199777,0.200055,0.200055,anticipation
2,Guess who always makes you smile? 😃,"[-0.13193609658628702, 0.20359431293659977, -0...",0.200057,0.200061,0.199765,0.200057,0.200059,anticipation
3,INCOMING:Max G.I.Joe Ninja action ft.Henry Gol...,"[-0.07782417975977296, 0.12250001124006563, -0...",0.200053,0.200061,0.199772,0.200057,0.200057,anticipation
4,New AC/TV/Laptop on your mind?,"[-0.19056224822998047, 0.297525935433805, -0.3...",0.200055,0.200062,0.199765,0.20006,0.200058,anticipation


In [466]:
vectors = vectors.reset_index(drop = True)

In [467]:
emails['prediction'].value_counts()

anticipation    1507
trust             17
pride             15
joy                2
Name: prediction, dtype: int64

In [468]:
from sklearn.cluster import KMeans
n_clusters = 15
clf = KMeans(n_clusters=n_clusters, max_iter=100, init='k-means++', n_init=1)
labels = clf.fit_predict(vectors)
centroids = clf.cluster_centers_

In [469]:
def abs_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += abs(value - centroid[index])
    return dist

def square_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += (value - centroid[index])**2
    return dist

In [470]:
emails['Cluster'] = labels
emails['abs_dist'] = ''
for index, vect in enumerate(emails['Vector']):
    cluster = emails['Cluster'].loc[index]
    centroid = centroids[cluster]
    emails['abs_dist'].loc[index] = abs_distance(vect, centroid)

C:\Users\Harsh Garg\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [471]:
min_indices = []
for x in range(0,15):
    min_indices.append(emails[emails['abs_dist']==min(emails[emails['Cluster']==x]['abs_dist'])].index)

In [472]:
for x in range(0,15):
    print(emails['Subject Line'].loc[min_indices[x][:]])

282    Here’s what your awesome week looks like!
Name: Subject Line, dtype: object
1299    Lessons From a Body Sculptor on What Really Ch...
Name: Subject Line, dtype: object
970    IITM: IRIS Webinar Series
Name: Subject Line, dtype: object
824    Warning! This is not a Price Drop
Name: Subject Line, dtype: object
1303    MSP of Rabi crops increased; Rs. 10,000 crore ...
Name: Subject Line, dtype: object
800    Good Health Essential #1: Oximeters\n\n
Name: Subject Line, dtype: object
466    LOVE Surprises?
467    LOVE Surprises?
776    LOVE Surprises?
Name: Subject Line, dtype: object
887    Something for the Women in your Life!
Name: Subject Line, dtype: object
1232    What’s going on with our 10-minute delivery?
Name: Subject Line, dtype: object
284    Schedule your manicure after watching these na...
Name: Subject Line, dtype: object
238    Crazy' surprises inside!
239    Crazy' surprises inside!
Name: Subject Line, dtype: object
64      Twinkle Twinkle SUPERST⭐R
65      Twinkle Tw

In [475]:
emails.to_excel(r"C:\Users\Harsh Garg\Desktop\intract\FastText_pretrained_lee_corpus.xlsx",index=False)

#### Trained on scrapped data

In [476]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

def create_dataset(keyword):
    scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/' + keyword)
    article = scrapped_data.read()

    parsed_article = bs.BeautifulSoup(article,'html.parser')

    paragraphs = parsed_article.find_all('p')

    article_text = ""

    for p in paragraphs:
        article_text += p.text

    # Cleaing the text
    processed_article = article_text.lower()
    processed_article = re.sub('[^a-zA-Z]', '', processed_article )
    processed_article = re.sub(r'\s+', ' ', processed_article)
    
    stopwords = ENGLISH_STOP_WORDS.union(['swiggy','zomato','aggarwal','harsh','garg','abhishek','2021','11','ve','ncr',
                                          'delhi','sep','october','india','anita','harshgarg297@gmail.com',
                                          'abhishek12318@gmail.com','000','10','100','12','13','14','143','16','19','20','2020',
                                          '2022','21','2192573831','24','25','250','2568642602','26th','30','360','3657963965',
                                          '420','499','50','500','60','75','80','90','99','999','abhishek12318', 'com', 
                                          'gmail', 'harshgarg297','ramji','kyc','ft','prathamesh','pathwardhan','singh','flipkart',
                                          'amazon','january','jan','feb','february','mar','march','apr','april','may','june','jun',
                                          'july','aug','august','sept','september','oct','nov','november','dec','december'])


    for i in range(len(all_words)):
        all_words[i] = [w for w in all_words[i] if w not in stopwords]
        
    return all_words[0]

In [477]:
dataset = []
dataset += create_dataset('emotions')
dataset += create_dataset('fear')
dataset += create_dataset('anticipation')
dataset += create_dataset('joy')
dataset += create_dataset('trust')
dataset += create_dataset('pride')

In [478]:
len(dataset)

27252

In [479]:
from gensim.models import FastText

model = FastText(vector_size=100, window=2, min_count=1)  # instantiate
model.build_vocab(dataset)
model.train(dataset, total_examples=len(dataset), epochs=10)  # train

(341007, 2048580)

In [480]:
def abs_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += abs(value - centroid[index])
    return dist

def square_distance(point, centroid):
    dist = 0
    for index, value in enumerate(point):
        dist += (value - centroid[index])**2
    return dist

In [481]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [482]:
def data_preprocess_and_preds(pred_model):
    
    #Import data
    emails = pd.read_excel(r"C:\Users\Harsh Garg\Desktop\intract\Email Subject Line Scrapper.xlsx",'Saved Data')

    #Delete irrelevant columns and add relevant columns
    emails = emails[['Subject Line']]
    
    emails['Vector'] = ''

    emails['fear'] = ''

    emails['anticipation'] = ''

    emails['joy'] = ''

    emails['trust'] = ''

    emails['pride'] = ''

    emails['prediction'] = ''

    #Will be required for using a 'Switch' function later on
    dictionary = {0:'fear',
                 1:'anticipation',
                 2:'joy',
                 3:'trust',
                 4:'pride'}
    
    #to store vectors of the words of emails for k-means later
    vectors = pd.DataFrame()
    
    #Predictions
    for index,line in enumerate(emails['Subject Line']):
        words = line.split()
        vector_word = [0]*100
        for word in words:
            vector_word += pred_model[word]
        vector_word /= len(words)
        emails['Vector'].loc[index] = vector_word
        vectors = pd.concat([vectors, pd.DataFrame(vector_word).T])
        probs = []
        probs.append(pred_model.n_similarity(words,['fear']))
        probs.append(pred_model.n_similarity(words,['anticipation']))
        probs.append(pred_model.n_similarity(words,['joy']))
        probs.append(pred_model.n_similarity(words,['trust']))
        probs.append(pred_model.n_similarity(words,['pride']))
        probs = softmax(probs)
        emails['fear'].loc[index] = probs[0]
        emails['anticipation'].loc[index] = probs[1]
        emails['joy'].loc[index] = probs[2]
        emails['trust'].loc[index] = probs[3]
        emails['pride'].loc[index] = probs[4]

        emails['prediction'].loc[index] = np.where(probs == max(probs))[0][0]
    emails['prediction'] = emails['prediction'].apply(lambda x: dictionary.get(x))

    vectors = vectors.reset_index(drop = True)

    emails['prediction'].value_counts()

    from sklearn.cluster import KMeans
    n_clusters = 15
    clf = KMeans(n_clusters=n_clusters, max_iter=100, init='k-means++', n_init=1)
    labels = clf.fit_predict(vectors)
    centroids = clf.cluster_centers_

    emails['Cluster'] = labels
    emails['abs_dist'] = ''
    for index, vect in enumerate(emails['Vector']):
        cluster = emails['Cluster'].loc[index]
        centroid = centroids[cluster]
        emails['abs_dist'].loc[index] = abs_distance(vect, centroid)

    min_indices = []
    for x in range(0,15):
        min_indices.append(emails[emails['abs_dist']==min(emails[emails['Cluster']==x]['abs_dist'])].index)
    
    return emails, vectors

In [483]:
mails, vecs = data_preprocess_and_preds(model.wv)

C:\Users\Harsh Garg\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [484]:
mails.shape,vecs.shape

((1541, 10), (1541, 100))

In [485]:
for x in range(0,15):
    print(mails['Subject Line'].loc[min_indices[x][:]])

282    Here’s what your awesome week looks like!
Name: Subject Line, dtype: object
1299    Lessons From a Body Sculptor on What Really Ch...
Name: Subject Line, dtype: object
970    IITM: IRIS Webinar Series
Name: Subject Line, dtype: object
824    Warning! This is not a Price Drop
Name: Subject Line, dtype: object
1303    MSP of Rabi crops increased; Rs. 10,000 crore ...
Name: Subject Line, dtype: object
800    Good Health Essential #1: Oximeters\n\n
Name: Subject Line, dtype: object
466    LOVE Surprises?
467    LOVE Surprises?
776    LOVE Surprises?
Name: Subject Line, dtype: object
887    Something for the Women in your Life!
Name: Subject Line, dtype: object
1232    What’s going on with our 10-minute delivery?
Name: Subject Line, dtype: object
284    Schedule your manicure after watching these na...
Name: Subject Line, dtype: object
238    Crazy' surprises inside!
239    Crazy' surprises inside!
Name: Subject Line, dtype: object
64      Twinkle Twinkle SUPERST⭐R
65      Twinkle Tw

In [486]:
mails[mails['prediction']=='trust']

,Subject Line,Vector,fear,anticipation,joy,trust,pride,prediction,Cluster,abs_dist
4,New AC/TV/Laptop on your mind?,"[0.0001063522882759571, 0.0009480616252403706,...",0.195452,0.192447,0.180315,0.225437,0.206349,trust,12,0.065703
5,New AC/TV/Laptop on your mind?,"[0.0001063522882759571, 0.0009480616252403706,...",0.195452,0.192447,0.180315,0.225437,0.206349,trust,12,0.065703
6,Make sure your meal meets your fitness goals. 💪🏻,"[0.001995351134408136, 0.0006653189710858795, ...",0.180146,0.190449,0.208112,0.227242,0.19405,trust,8,0.049649
10,Your Swiggy order was delivered superfast!,"[0.000664175187315171, 0.0007133712812598484, ...",0.171943,0.194315,0.209483,0.223443,0.200816,trust,8,0.042764
11,Put month-end blues on snooze ⏰,"[0.0009930937376338989, 0.0016646496551402379,...",0.17102,0.201793,0.203777,0.226361,0.197049,trust,0,0.10474
...,...,...,...,...,...,...,...,...,...,...
1516,The July highlights on Nintendo Switch,"[0.0014786439811966072, 0.000670435544331364, ...",0.187761,0.207598,0.177065,0.224682,0.202893,trust,12,0.062502
1518,"""PSA for DBS Broly players: Medium and EX gra...""","[0.00021032798556714423, -5.262130111481788e-0...",0.209273,0.209134,0.191791,0.212652,0.177149,trust,8,0.055817
1534,Party with DJ Tarun and DJ Shreya,"[-0.0003432029869665192, -0.000619665162438260...",0.19647,0.164867,0.213016,0.215098,0.210549,trust,8,0.088762
1535,3 Essential Guitar Scales,"[-0.0003252516617067158, -0.002024852299655322...",0.202556,0.203229,0.205574,0.209002,0.179639,trust,8,0.129149


In [489]:
mails['prediction'].value_counts()

fear            401
anticipation    394
trust           261
joy             258
pride           227
Name: prediction, dtype: int64

In [490]:
mails.to_excel(r"C:\Users\Harsh Garg\Desktop\intract\FastText_custom_trained_wikipedia_5_emotions_scraped.xlsx",index=False)